## Импорты

In [1]:
from itertools import product
from pprint import pprint
from typing import Any

import pandas as pd
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

from datasets import download_dataset_from_kaggle
from preprocessing import (
    delete_stop_words,
    lemmatize,
    normalize,
    tokenize_by_sentence,
    tokenize_by_words,
)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
RANDOM_STATE = 42

## Загружаем датасет 

In [3]:
dataset_path = download_dataset_from_kaggle(
    "ozlerhakan/spam-or-not-spam-dataset", "spam_or_not_spam"
)

In [4]:
df = pd.read_csv(f"{dataset_path}/spam_or_not_spam.csv")

## EDA

Посмотрим, что есть в датасете, уберем nan значения, если они есть

In [5]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [6]:
df.describe()

,label
count,3000.000000
mean,0.166667
std,0.372740
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
df["email"].isnull().sum()

1

In [8]:
df.dropna(inplace=True)

## Препроцессинг данных

In [9]:
def preprocess(text: str, lang: str) -> str:
    sentences: str = tokenize_by_sentence(text, lang)
    preprocessed_sentences: list[str] = []
    for sentence in sentences:
        normalized_text = normalize(sentence, lang)
        tokens = tokenize_by_words(normalized_text, lang)
        clean_tokens = delete_stop_words(tokens, lang)
        lemmatized_tokens = lemmatize(clean_tokens)
        preprocessed_sentences.append(" ".join([token for token in lemmatized_tokens]))
    return " ".join(preprocessed_sentences)

In [10]:
df["email"] = df["email"].apply(lambda x: preprocess(x, "eng"))

In [11]:
df["email"].head()

0    date wed number aug number number number numbe...
1    martin posted tassos papadopoulos greek sculpt...
2    man threatens explosion moscow thursday august...
3    klez virus die already prolific virus ever kle...
4    adding cream spaghetti carbonara effect pasta ...
Name: email, dtype: object

## Способы векторизации

Рассмотрим TfidfVectorizer и CountVectorizer из sklearn как способы векторизации текста

In [12]:
def show_vectorizer_info(
    vectorizer, text: list[str] | pd.Series | pd.DataFrame
) -> None:
    embedding = pd.DataFrame(
        vectorizer.fit_transform(text).toarray(),
        columns=vectorizer.get_feature_names_out(),
    )
    print(f"Vectorizer class {vectorizer.__class__}\n")
    print(f"Vectorizer embedding: {embedding}\n")
    print(f"Vocabulary info: {vectorizer.vocabulary_}\n")

In [13]:
show_vectorizer_info(CountVectorizer(), df["email"].iloc[:2])

Vectorizer class <class 'sklearn.feature_extraction.text.CountVectorizer'>

Vectorizer embedding:    able  actually  admiring  ago  alexander  amphitheatre  athos  aug  behind  \
0     1         1         0    1          0             0      0    1       0   
1     0         0         1    0          1             1      1    0       1   

   car  ...  version  vircio  weather  wed  well  wide  without  workers  \
0    0  ...        3       1        0    1     0     0        1        2   
1    1  ...        0       0        1    0     1     1        0        0   

   works  yahoo  
0      1      0  
1      0      2  

[2 rows x 145 columns]

Vocabulary info: {'date': 21, 'wed': 138, 'number': 87, 'aug': 7, 'chris': 10, 'garrigues': 45, 'cwg': 20, 'dated': 22, 'numberfanumberd': 88, 'deepeddy': 25, 'com': 12, 'message': 78, 'id': 57, 'tmda': 129, 'vircio': 136, 'reproduce': 107, 'error': 30, 'repeatable': 105, 'like': 67, 'every': 31, 'time': 127, 'without': 141, 'fail': 36, 'debug': 24

In [14]:
show_vectorizer_info(TfidfVectorizer(), df["email"].iloc[:2])

Vectorizer class <class 'sklearn.feature_extraction.text.TfidfVectorizer'>

Vectorizer embedding:        able  actually  admiring       ago  alexander  amphitheatre     athos  \
0  0.028104  0.028104  0.000000  0.028104   0.000000      0.000000  0.000000   
1  0.000000  0.000000  0.104165  0.000000   0.104165      0.104165  0.104165   

        aug    behind       car  ...   version    vircio   weather       wed  \
0  0.028104  0.000000  0.000000  ...  0.084311  0.028104  0.000000  0.028104   
1  0.000000  0.104165  0.104165  ...  0.000000  0.000000  0.104165  0.000000   

       well      wide   without   workers     works     yahoo  
0  0.000000  0.000000  0.028104  0.056207  0.028104  0.000000  
1  0.104165  0.104165  0.000000  0.000000  0.000000  0.208331  

[2 rows x 145 columns]

Vocabulary info: {'date': 21, 'wed': 138, 'number': 87, 'aug': 7, 'chris': 10, 'garrigues': 45, 'cwg': 20, 'dated': 22, 'numberfanumberd': 88, 'deepeddy': 25, 'com': 12, 'message': 78, 'id': 57, 'tmda': 

Вывод: и CountVectorizer, и TfidfVectorizer в себя хранят одинаковые словари-счетчики слов, но у них отличается представление эмбеддингов - CountVectorizer выдает просто частоту встречаемости слов, а TfidfVectorizer отражает важность слова для документа в корпусе.

## Обучаем модели с подбором гиперпараметров

Разобьем выборку на train и test подвыборки

In [15]:
X = df["email"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [16]:
def grid_search_params(vectorizer, classifier) -> dict[str, Any]:
    """Создает пайплайн для векторизатора и классификатора и подбирает гиперпараметры для этого пайплайна."""
    pipeline = Pipeline(
        [("vectorizer", vectorizer["model"]), ("classifier", classifier["model"])]
    )
    param_grid = {
        f"vectorizer__{key}": value for key, value in vectorizer["param_grid"].items()
    }
    param_grid.update(
        {f"classifier__{key}": value for key, value in classifier["param_grid"].items()}
    )
    grid_search = HalvingGridSearchCV(
        pipeline, param_grid, cv=5, random_state=RANDOM_STATE
    )
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    test_score = best_model.score(X_test, y_test)
    return {
        "Vectorizer": vectorizer["model"].__class__.__name__,
        "Classifier": classifier["model"].__class__.__name__,
        "Лучшие гиперпараметры": grid_search.best_params_,
        "Лучшая оценка на тестовой выборке": test_score,
    }

Создадим список всех векторизаторов

In [17]:
vectorizers = [
    {
        "model": TfidfVectorizer(),
        "param_grid": {
            "max_features": [100, 500, 1000],
            "norm": ["l1", "l2"],
        },
    },
    {
        "model": CountVectorizer(),
        "param_grid": {"ngram_range": [(1, 1), (1, 2), (1, 3)]},
    },
]

Создадим список всех классификаторов

In [18]:
classifiers = [
    {
        "model": DecisionTreeClassifier(random_state=RANDOM_STATE),
        "param_grid": {
            "max_depth": [3, 5, 7],
            "criterion": ["gini", "log_loss", "entropy"],
        },
    },
    {
        "model": LogisticRegression(solver="liblinear", random_state=RANDOM_STATE),
        "param_grid": {"C": [0.1, 1, 10], "penalty": ["l1", "l2"]},
    },
    {"model": BernoulliNB(), "param_grid": {"alpha": [0.1, 1, 10]}},
]

In [19]:
experiments = list(product(vectorizers, classifiers))

Запускаем поиск гиперпараметров по всем экспериментам

In [20]:
for vectorizer, classifier in tqdm(experiments):
    pprint(grid_search_params(vectorizer=vectorizer, classifier=classifier))

 17%|███████████████████████████████▌                                                                                                                                                             | 1/6 [00:22<01:53, 22.69s/it]

{'Classifier': 'DecisionTreeClassifier',
 'Vectorizer': 'TfidfVectorizer',
 'Лучшая оценка на тестовой выборке': 0.95,
 'Лучшие гиперпараметры': {'classifier__criterion': 'log_loss',
                           'classifier__max_depth': 5,
                           'vectorizer__max_features': 100,
                           'vectorizer__norm': 'l2'}}


 33%|███████████████████████████████████████████████████████████████                                                                                                                              | 2/6 [00:36<01:11, 17.76s/it]

{'Classifier': 'LogisticRegression',
 'Vectorizer': 'TfidfVectorizer',
 'Лучшая оценка на тестовой выборке': 0.9866666666666667,
 'Лучшие гиперпараметры': {'classifier__C': 10,
                           'classifier__penalty': 'l2',
                           'vectorizer__max_features': 500,
                           'vectorizer__norm': 'l2'}}


 50%|██████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                              | 3/6 [00:49<00:46, 15.36s/it]

...

{'Classifier': 'DecisionTreeClassifier',
 'Vectorizer': 'CountVectorizer',
 'Лучшая оценка на тестовой выборке': 0.95,
 'Лучшие гиперпараметры': {'classifier__criterion': 'gini',
                           'classifier__max_depth': 5,
                           'vectorizer__ngram_range': (1, 2)}}


 83%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                               | 5/6 [02:01<00:28, 28.96s/it]

{'Classifier': 'LogisticRegression',
 'Vectorizer': 'CountVectorizer',
 'Лучшая оценка на тестовой выборке': 0.99,
 'Лучшие гиперпараметры': {'classifier__C': 10,
                           'classifier__penalty': 'l1',
                           'vectorizer__ngram_range': (1, 3)}}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:14<00:00, 22.47s/it]

{'Classifier': 'BernoulliNB',
 'Vectorizer': 'CountVectorizer',
 'Лучшая оценка на тестовой выборке': 0.9716666666666667,
 'Лучшие гиперпараметры': {'classifier__alpha': 0.1,
                           'vectorizer__ngram_range': (1, 1)}}


Вывод: среди всех экспериментов самое лучшее сочетание моделей следующее:

Classifier: LogisticRegression  
Vectorizer: CountVectorizer  
Лучшая оценка на тестовой выборке: 0.99  
Лучшие гиперпараметры:  
* classifier__C: 10  
* classifier__penalty: 'l1',  
* vectorizer__ngram_range: (1, 3)}  